# Example for building a model extension

Let's assume we want to use the DEBBase model and add some more detail in a specific part of the model. <br>
One way to do this is to clone the DEBBase repository and start modifying the source code. <br>

That is a valid way to approach things, but during the development process, we might however not always want to create a new repository whenever we test a slightly different model structure. <br>

Also, it would be nice to automatically have a visual indication in the code that shows us which parts of the model have changed, or to directly have modifications documented in a notebook if we only wish to perform some *ad hoc* tests of different model structures. <br>

This notebook demonstrates how to implement such *ad hoc* modifications without creating an entirely new implementation. There is nothing revolutionary about it and it is still a bit of coding work, but to have a notebook at the end of this process which documents the changes without the need of creating a heap of new files.

## Setup

In [52]:
using Pkg; Pkg.activate("../test")

  Activating project at `c:\Users\simon\Documents\Julia\DEBBase.jl\test`


In [53]:
using DEBBase.DEBODE
using DEBBase.ParamStructs

## Implementing the derivative function

In our extended model, organisms change their energy allocation as food availability decreases (phenotypic plasticity). <br>
Also, it is part of the ODE system, rather than the rule-based additions of the ABM. <br>

To modify the ODE, the first step is to familiarize ourselves with the source file `derivatives.jl`. <br>

Upon inspecting this file, we see that the core of the model is put together in the function `DEBODE_agent_IA!` (agent because these derivatives are applied for each agent at every timestep in the ABM, IA because the model assumes independent action for mixture effects).

In [54]:
less(DEBODE.DEBODE_agent_IA!)

@inline function DEBODE_agent_IA!(du, u, p, t)
    
    dD!(du, u, p, t) # change in damage
    y_z_IA!(du, u, p, t) # response to chemical stressors
    tempcorr!(du, u, p, t) # response to temperature
    apply_stressors!(du, u, p, t) # apply all stressor / environmental effects to baseline parameters

    #### auxiliary state variables (record cumulative values)
    dI!(du, u, p, t)
    dA!(du, u, p, t) 
    dM!(du, u, p, t) 
    dJ!(du, u, p, t)
    #dQ!(du, u, p, t)# dissipation flux is currently not used - outcommented since this currently does memory allocations

    #### major state variables
    dS!(du, u, p, t) # structures
    dS_max_hist!(du, u, p, t) # reference structure
    dH!(du, u, p, t) # maturity 
    dH_b!(du, u, p, t) # estimate of maturity at birth
    dR!(du, u, p, t) # reproduction buffer

    return nothing
end

@inline function DEBODE_IA!(du, u, p, t)
    DEBODE_global!(du, u, p, t)
    DEBODE_agent_IA!(du, u, p, t)
end


Given the structure of this function, it makes sense to first define a small function that calculates the relative response in the resource allocation parameter $\kappa$ as a function of the scaled relative response $f_X$.

In [103]:
function adaptive_plasticity!(du, u, p, t)::Nothing
    # this is a very naive way to implement this...I don't recommend to actually use this relationship
    u.y_adpt = min(1, -p.spc.slope_adpt + 1)
    return nothing # the function only modifies u - no return value
end

adaptive_plasticity! (generic function with 1 method)

The `apply_stressors!` function is a good place to perform this kind modification. <br>
This is where baseline parameters are changed, and placing the effect of adaptive plasticity elsewhere could mean that we overwrite the effect of another stressor (would not be the case in this particular case, but it is a good idea to keep it in one place). <br>

This is then our modified code for `apply_stressors!`.

In [104]:
function apply_stressors!(du, u, p, t)::Nothing
    u.eta_AS = p.spc.eta_AS_0 * u.y_G 
    u.k_M = p.spc.k_M_0 * u.y_M * u.y_T
    u.k_J = p.spc.k_J_0 * u.y_M * u.y_T
    u.eta_IA = p.spc.eta_IA_0 * u.y_A
    u.eta_AR = p.spc.eta_AR_0 * u.y_R

    u.Idot_max_rel_emb = p.agn.Idot_max_rel_emb_0 * u.y_T

    u.kappa = p.spc.kappa_0 * u.y_adpt

    return nothing
end

apply_stressors! (generic function with 1 method)

And the re-defined ODE system:

In [105]:
@inline function DEBODE_agent_IA_adpt!(du, u, p, t)
    
    DEBODE.dD!(du, u, p, t) # change in damage
    DEBODE.y_z_IA!(du, u, p, t) # response to chemical stressors
    DEBODE.tempcorr!(du, u, p, t) # response to temperature
    Main.apply_stressors!(du, u, p, t) # apply all stressor / environmental effects to baseline parameters

    #### auxiliary state variables (record cumulative values)
    DEBODE.dI!(du, u, p, t)
    DEBODE.dA!(du, u, p, t) 
    DEBODE.dM!(du, u, p, t) 
    DEBODE.dJ!(du, u, p, t)
    #dQ!(du, u, p, t)# dissipation flux is currently not used - outcommented since this currently does memory allocations

    #### major state variables
    DEBODE.dS!(du, u, p, t) # structures
    DEBODE.dS_max_hist!(du, u, p, t) # reference structure
    DEBODE.dH!(du, u, p, t) # maturity 
    DEBODE.dH_b!(du, u, p, t) # estimate of maturity at birth
    DEBODE.dR!(du, u, p, t) # reproduction buffer

    return nothing
end

@inline function DEBODE_IA_adpt!(du, u, p, t)
    DEBODE.DEBODE_global!(du, u, p, t)
    DEBODE_agent_IA_adpt!(du, u, p, t)
end


DEBODE_IA_adpt! (generic function with 1 method)

Note the change in the source code compared to the original version: <br>
We have a `DEBODE.` in front of every function that we have *not* changed, immediately giving us an indication of what *has* changed.

## Adding new state variables and parameters 

Running the model now would throw an error, because we have introduced two new parameters and one new state variable which have not been defined yet. <br>
State variables in the ODE are given as `ComponentVector`. <br>
Specifically, we have a function `initialize_agent_statevars` (the separation from `initialize_global_statevars` is needed for the ABM). <br>

We can define a new function `initialize_agent_statevars_adpt` that concatenates our new state variable to those which are already defined.

In [106]:
function initialize_agent_statevars_adpt(p)
    vcat(
        DEBODE.initialize_agent_statevars(p),
        ComponentVector(y_adpt = 1.)
    )
end

initialize_agent_statevars_adpt (generic function with 1 method)

For the parameters, things are bit more complicated, because the parameters are stored in `mutable struct`s and these are not as easily extensible (by design). <br>
We have two obvious ways forward here: Either convert the parameters to a NamedTuple using NamedTupleTools,...

In [109]:
using NamedTupleTools

spc = SpeciesParams() |> ntfromstruct
spc_adpt = (;
    spc...,
    (
        slope_adpt = 0.5,
    )...
)

p_adpt = (
    glb = GlobalParams() |> ntfromstruct,
)

(N0 = 1, t_max = 21.0, Xdot_in = 1200.0, k_V = 0.0, V_patch = 0.05, C_W = [0.0], T = 293.15)

In [ ]:
p = 

In [ ]:
@with_kw mutable struct SpeciesParams <: AbstractSpeciesParams
    Z::Distribution = Dirac(1.) # agent variability is accounted for in the zoom factor. This can be set to a Dirac distribution if a zoom factor should be applied without introducing agent variability.
    propagate_zoom::@NamedTuple{
        Idot_max_rel_0, 
        Idot_max_rel_emb_0, 
        X_emb_int_0::Bool, 
        H_p_0::Bool, 
        # Parameters to which Z will be propagated
        # individual_variability! will take care of appropriate scaling
        K_X_0::Bool} = ( 
            Idot_max_rel_0 = true, 
            Idot_max_rel_emb_0 = true,
            X_emb_int_0 = true,
            H_p_0 = true, 
            K_X_0 = true
        )         
    T_A::Float64 = 8000. # Arrhenius temperature [K]
    T_ref::Float64 = 293.15 # reference temperature [K]
    X_emb_int_0::Float64 = 19.42 # initial vitellus [μgC]
    K_X_0::Float64 = 10_000 # half-saturation constant for food uptake [μgC L-1]
    Idot_max_rel_0::Float64 = 22.9 # maximum size-specific ingestion rate [μgC μgC^-(2/3) d-1]
    Idot_max_rel_emb_0::Float64 = 22.9 # size-specific embryonic ingestion rate [μgC μgC^-(2/3) d-1]
    kappa_0::Float64 = 0.539 # somatic allocation fraction [-]
    eta_IA_0::Float64 = 0.33 # assimilation efficiency [-]
    eta_AS_0::Float64 = 0.8 # growth efficiency [-]
    eta_SA::Float64 = 0.8 # shrinking efficiency [-]
    eta_AR_0::Float64 = 0.95 # reproduction efficiency [-]
    k_M_0::Float64 = 0.59 # somatic maintenance rate constant [d^-1]
    k_J_0::Float64 = 0.504 # maturity maintenance rate constant [d^-1]
    H_p_0::Float64 = 100. # maturity at puberty [μgC]
    
    k_D_G::Vector{Float64} = [0.00] # toxicokinetic rate constants | PMoA growth efficiency
    k_D_M::Vector{Float64} = [0.00] # toxicokinetic rate constants | PMoA maintenance costs
    k_D_A::Vector{Float64} = [0.00] # toxicokinetic rate constants | PMoA assimilation efficiency
    k_D_R::Vector{Float64} = [0.38] # toxicokinetic rate constants | PMoA reproduction efficiency
    k_D_h::Vector{Float64} = [0.00] # toxicokinetic rate constants | PMoA hazard rate
    
    drc_functs_G::Vector{Function} = [LL2]  # dose-response functions | PMoA growth efficiency
    drc_functs_M::Vector{Function} = [LL2M] # dose-response functions | PMoA maintenance costs
    drc_functs_A::Vector{Function} = [LL2]  # dose-response functions | PMoA assimilation efficiency
    drc_functs_R::Vector{Function} = [LL2]  # dose-response functions | PMoA reproduction efficiency
    drc_functs_h::Vector{Function} = [LL2h] # dose-response functions | PMoA hazard rate

    e_G::Union{Nothing,Vector{Float64}} = [1e10] # sensitivity parameters | PMoA growth efficiency
    e_M::Union{Nothing,Vector{Float64}} = [1e10] # sensitivity parameters | PMoA maintenance costs
    e_A::Union{Nothing,Vector{Float64}} = [1e10] # sensitivity parameters | PMoA assimilation efficiency
    e_R::Union{Nothing,Vector{Float64}} = [167.] # sensitivity parameters | PMoA reproduction efficiency
    e_h::Union{Nothing,Vector{Float64}} = [1e10] # sensitivity parameters | PMoA hazard rate

    b_G::Union{Nothing,Vector{Float64}} = [1e10] # slope parameters | PMoA growth efficiency
    b_M::Union{Nothing,Vector{Float64}} = [1e10] # slope parameters | PMoA maintenance costs
    b_A::Union{Nothing,Vector{Float64}} = [1e10] # slope parameters | PMoA assimilation efficiency
    b_R::Union{Nothing,Vector{Float64}} = [0.93] # slope parameters | PMoA reproduction efficiency
    b_h::Union{Nothing,Vector{Float64}} = [1e10] # slope parameters | PMoA reproduction efficiency

    # the following are parameters which are currently only relevant for the  ABM
    
    f_Xthr::Float64 = 0.5 # functional response threshold for starvation mortality
    s_min::Float64 = 0.25 # daily survival mortality at complete food deprivation

    a_max = Truncated(Normal(60, 6), 0, Inf) # maximum life span 
    tau_R = 2. # reproduction interval
end

In [72]:
function simulator(
    params::Union{AbstractParamCollection,NamedTuple}; 
    alg = Tsit5(),
    saveat = 1,
    reltol = 1e-6,
    model = DEBODE_IA!,
    AgentParamType::DataType = AgentParams,
    kwargs...
    )::DataFrame

    params.agn = AgentParamType(params.spc) # initialize agent parameters incl. individual variability
    callbacks = lifestage_callbacks()

    u = initialize_statevars(params)
    prob = ODEProblem(model, u, (0, params.glb.t_max), params) # define the problem
    sol = solve(prob, alg; callback = callbacks, saveat = saveat, reltol = reltol, kwargs...) # get solution to the IVP
    simout = sol_to_df(sol) # convert solution to dataframe

    return simout
end



ErrorException: type NamedTuple has no field y_adpt

Now we would be ready to, for example, fit our modified model to some data and compare it with the default model. <br>
As you see, it is still quite a bit of work 

In [23]:
DEBODE_agent_IA!

DEBODE_agent_IA! (generic function with 1 method)